<h1 style="background: linear-gradient(180deg,rgb(92, 0, 128) 0%,rgb(46, 0, 153) 75%, rgb(65, 0, 230) 100%); color: white; font-family: 'Raleway', sans-serif; padding: 10px 20px; border-radius: 10px; text-align: center; font-weight:500;">
Normalización
</h1>
<br>

**PRESENTAN** Armando Islas, Daniela Flores, Oscar Berrueco, Natalia Anaya

In [ ]:
import os
import re
import sys
import pandas as pd
import nltk
import emoji
import logging
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

nltk.download('punkt_tab')

In [ ]:
# Inicializar recursos de NLTK
stemmer = SnowballStemmer('spanish')
stop_words = set(stopwords.words('spanish'))

# Crear directorios de salida si no existen
OUTPUT_DIR = "../outputs/normalized/"
os.makedirs(OUTPUT_DIR, exist_ok=True)

In [16]:
def limpiar_texto_basico(texto):
    
    if not isinstance(texto, str):
        return ""  
    # Eliminar URLs
    texto = re.sub(r'https?://\S+|www\.\S+', '', texto)

    # Eliminar menciones (@usuario)
    texto = re.sub(r'@\w+', '', texto)
    
    # Eliminar emojis
    texto = emoji.replace_emoji(texto, replace='')
    
    # Eliminar símbolos irrelevantes pero conservar signos importantes para el clickbait
    texto = re.sub(r'[^\w\s\?\!\.\,\:\;\…]', '', texto)
    
    # Eliminar espacios múltiples
    texto = re.sub(r'\s+', ' ', texto).strip()
    
    return texto

In [17]:
def tokenizar_texto(texto):
    
    if not isinstance(texto, str):
        return []
    
    return word_tokenize(texto)

In [18]:
def eliminar_stopwords(tokens):
    
    return [token for token in tokens if token.lower() not in stop_words]

In [19]:
def lematizar_texto(texto):
    
    if not isinstance(texto, str):
        return []
    
    doc = nlp(texto)
    return [token.lemma_ for token in doc]

In [20]:
def aplicar_stemming(tokens):

    return [stemmer.stem(token) for token in tokens]

In [21]:
def normalizar_token_stop(texto):
    
    tokens = tokenizar_texto(texto)
    tokens_sin_stop = eliminar_stopwords(tokens)
    return ' '.join(tokens_sin_stop)

In [22]:
def normalizar_token_stop_lemma(texto):
    
    lemas = lematizar_texto(texto)
    lemas_sin_stop = [lema for lema in lemas if lema.lower() not in stop_words]
    return ' '.join(lemas_sin_stop)

In [23]:
def normalizar_all(texto):

    texto_limpio = limpiar_texto_basico(texto)
    lemas = lematizar_texto(texto_limpio)
    lemas_sin_stop = [lema for lema in lemas if lema.lower() not in stop_words]
    stems = aplicar_stemming(lemas_sin_stop)
    return ' '.join(stems)

In [24]:
def procesar_dataset(input_path):
    
    try:
        df = pd.read_csv(input_path)
    except Exception as e:
        return {}
    
    resultados = {}
    
    # Verificar si la columna existe
    if 'Teaser Text' not in df.columns:
        return {}
    
    # Aplicar técnica 1: tokenización + eliminación de stopwords
    df_token_stop = df.copy()
    df_token_stop['Teaser Text'] = df_token_stop['Teaser Text'].apply(normalizar_token_stop)
    resultados['token_stop'] = df_token_stop
    
    # Aplicar técnica 2: tokenización + stopwords + lematización
    df_token_stop_lemma = df.copy()
    df_token_stop_lemma['Teaser Text'] = df_token_stop_lemma['Teaser Text'].apply(normalizar_token_stop_lemma)
    resultados['token_stop_lemma'] = df_token_stop_lemma
    
    # Aplicar técnica 3: todas las técnicas
    df_all = df.copy()
    df_all['Teaser Text'] = df_all['Teaser Text'].apply(normalizar_all)
    resultados['all'] = df_all
    
    return resultados

In [25]:
def guardar_resultados(resultados):
    for tecnica, df in resultados.items():
        output_path = os.path.join(OUTPUT_DIR, f"normalized_{tecnica}.csv")
        try:
            df.to_csv(output_path, index=False)
        except:
            print('No se pudo guardar el archivo')

In [26]:
input_path = "../data/TA1C_dataset_detection_train.csv"

# Verificar si el archivo existe
if not os.path.exists(input_path):
    sys.exit(1)

resultados = procesar_dataset(input_path)

if resultados:
    guardar_resultados(resultados)